In [ ]:
!which python

In [ ]:
import numpy as np 
import pandas as pd 
import osmnx as ox 
from shapely.wkt import loads 
import geopandas as gpd 
import matplotlib.pyplot as plt 

In [ ]:
survey_area = gpd.read_file('demand_inputs/S05-a-10_SYUTO_GML/S05-a-10_SYUTO-g_Occurred_ConcentratedTrafficVolumeOfPersonTrip.shp')
survey_area = survey_area.set_crs(4326)
print(survey_area.to_crs(3857).unary_union.area)
survey_area_boundary = gpd.GeoSeries(survey_area.unary_union.convex_hull)
survey_area_boundary.plot()
plt.show()

In [ ]:
### need to revise the "drivable" custom filter as part of highway 1 is labled as 'motor_vehicle = no'
custom_filter=(f'["highway"]["area"!~"yes"]'
        f'["highway"!~"abandoned|bridleway|construction|corridor|cycleway|elevator|'
        f"escalator|footway|path|pedestrian|planned|platform|proposed|raceway|service|"
        f'steps|track"]'
        # f'["motor_vehicle"!~"no"]["motorcar"!~"no"]'
        f'["service"!~"alley|driveway|emergency_access|parking|parking_aisle|private"]')
tokyo_graph_large = ox.graph_from_polygon(survey_area_boundary.iloc[0], network_type='all', custom_filter=custom_filter)
tokyo_nodes_large, tokyo_edges_large = ox.graph_to_gdfs(tokyo_graph_large)
tokyo_graph_component = ox.utils_graph.get_largest_component(tokyo_graph_large, strongly=True)
tokyo_nodes_component, tokyo_edges_component = ox.graph_to_gdfs(tokyo_graph_component)

In [4]:
tokyo_nodes, tokyo_edges = ox.graph_to_gdfs(tokyo_graph_component)
print(tokyo_nodes.shape, tokyo_edges.shape)

(1246321, 6) (3469474, 20)


In [7]:
tokyo_nodes.to_csv('network_inputs/tokyo_nodes_osmnx_raw.csv', index=False)
tokyo_edges.to_csv('network_inputs/tokyo_edges_osmnx_raw.csv', index=False)

In [8]:
tokyo_nodes_raw = pd.read_csv('network_inputs/tokyo_nodes_osmnx_raw.csv')
tokyo_edges_raw = pd.read_csv('network_inputs/tokyo_edges_osmnx_raw.csv')
display(tokyo_nodes_raw.head(3))
display(tokyo_edges_raw.head(3))

,y,x,osmid,highway,ref,geometry
0,35.026838,139.851108,131938557,traffic_signals,NaN,POINT (139.851108 35.0268381)
1,34.985095,139.849577,131939305,NaN,NaN,POINT (139.8495767 34.9850952)
2,34.984350,139.841655,131939336,traffic_signals,NaN,POINT (139.8416552 34.9843497)


,osmid,highway,oneway,length,geometry,lanes,ref,name,bridge,maxspeed,tunnel,width,access,junction,area,service,est_width,u,v,key
0,13945297,tertiary,False,51.500,"LINESTRING (139.851108 35.0268381, 139.8510964...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131938557,6266083707,0
1,407006520,secondary,False,45.162,"LINESTRING (139.851108 35.0268381, 139.8512008...",2,302,館山富浦線,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131938557,305418247,0
2,407006520,secondary,False,184.183,"LINESTRING (139.851108 35.0268381, 139.8510121...",2,302,館山富浦線,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131938557,2466347437,0


In [9]:
np.sum(tokyo_edges_raw['length'])
print(np.sum(tokyo_edges_raw['oneway']==True), np.sum(tokyo_edges_raw['oneway']==False), tokyo_edges_raw.shape)
print(np.sum(tokyo_edges_raw.loc[tokyo_edges_raw['oneway']==True, 'length'])+np.sum(tokyo_edges_raw.loc[tokyo_edges_raw['oneway']==False, 'length'])/2)

126581 3244589 (3469474, 20)
3366303
157359311.89150003


In [10]:
# reformat the columns
tokyo_nodes = tokyo_nodes_raw.copy()
tokyo_nodes['lon'] = tokyo_nodes['x']
tokyo_nodes['lat'] = tokyo_nodes['y']
tokyo_nodes['node_id'] = np.arange(tokyo_nodes.shape[0])
tokyo_nodes = tokyo_nodes[['node_id', 'osmid', 'lon', 'lat']].reset_index(drop=True)
node_osmid_id_dict = {getattr(n, 'osmid'): getattr(n,'node_id') for n in tokyo_nodes.itertuples()}
display(tokyo_nodes.head(3))

,node_id,osmid,lon,lat
0,0,131938557,139.851108,35.026838
1,1,131939305,139.849577,34.985095
2,2,131939336,139.841655,34.984350


In [13]:
tokyo_edges = tokyo_edges_raw[['u', 'v', 'highway', 'length', 'lanes', 'maxspeed', 'geometry']].copy()
tokyo_edges['start_nid'] = tokyo_edges['u'].map(node_osmid_id_dict)
tokyo_edges['end_nid'] = tokyo_edges['v'].map(node_osmid_id_dict)

lanes_dict = {'motorway': 2, 'motorway_link': 2, 'trunk': 2, 'trunk_link': 2, 'primary': 2, 'primary_link': 2}
def fill_lanes(value):
    lanes = []
    if '[' in value: 
        value = value[1:-1].replace("'", "").replace(' ', '').split(',')
    else: value=[value]
    for v in value:
        if v in lanes_dict: lanes.append(lanes_dict[v])
        else: lanes.append(1)
    return(np.mean(lanes))

maxspeed_dict = {'motorway': 100, 'motorway_link': 100, 'trunk': 100, 'trunk_link': 100, 'primary': 60, 'primary_link': 60, 'secondary': 60, 'secondary_link': 60}
def fill_maxspeed(value):
    maxspeed = []
    if '[' in value: 
        value = value[1:-1].replace("'", "").replace(' ', '').split(',')
    else: value=[value]
    for v in value:
        if v in maxspeed_dict: maxspeed.append(maxspeed_dict[v])
        else: maxspeed.append(40)
    return(np.mean(maxspeed))

signal_dict = {'motorway': 1, 'motorway_link': 1, 'trunk': 1, 'trunk_link': 1}
def fill_signal(value):
    signal = []
    if '[' in value: 
        value = value[1:-1].replace("'", "").replace(' ', '').split(',')
    else: value=[value]
    for v in value:
        if v in signal_dict: signal.append(signal_dict[v])
        else: signal.append(0.5)
    return(np.max(signal))

def fill_type(value):
    if '[' in value: return 'multiple'
    else: return value

tokyo_edges['lanes'] = tokyo_edges['highway'].apply(fill_lanes)
tokyo_edges['maxspeed'] = tokyo_edges['highway'].apply(fill_maxspeed)
tokyo_edges['signal'] = tokyo_edges['highway'].apply(fill_signal)
tokyo_edges['type'] = tokyo_edges['highway'].apply(fill_type)
tokyo_edges['capacity'] = tokyo_edges['lanes']*1800*tokyo_edges['signal']
tokyo_edges['fft'] = tokyo_edges['length']/tokyo_edges['maxspeed']*3.6

display(tokyo_edges.head(3))
display(tokyo_edges.groupby('lanes').size())
display(tokyo_edges.groupby('maxspeed').size())
display(tokyo_edges.groupby('signal').size())
display(tokyo_edges.groupby('type').size())

,u,v,highway,length,lanes,maxspeed,geometry,start_nid,end_nid,signal,type,capacity,fft
0,131938557,6266083707,tertiary,51.500,1.0,40.0,"LINESTRING (139.851108 35.0268381, 139.8510964...",0,14135,0.5,tertiary,900.0,4.63500
1,131938557,305418247,secondary,45.162,1.0,60.0,"LINESTRING (139.851108 35.0268381, 139.8512008...",0,184,0.5,secondary,900.0,2.70972
2,131938557,2466347437,secondary,184.183,1.0,60.0,"LINESTRING (139.851108 35.0268381, 139.8510121...",0,8093,0.5,secondary,900.0,11.05098


lanes
1.000000    3312852
1.333333          2
1.500000         97
2.000000     156523
dtype: int64

maxspeed
40.000000     3218639
46.666667           8
50.000000         207
60.000000      194953
70.000000          23
80.000000          14
100.000000      55630
dtype: int64

signal
0.5    3413807
1.0      55667
dtype: int64

type
bus_guideway             2
bus_stop                 2
crossing                24
escape                   2
living_street        17207
motorway              1971
motorway_link         2668
multiple             17218
passing_place            2
primary              98893
primary_link          1989
residential        1537943
rest_area               26
road                   928
route                    2
secondary            93400
secondary_link         649
tertiary            397768
tertiary_link         1665
traffic_signals          4
trail                    4
trunk                48425
trunk_link            2518
unclassified       1246124
yes                     40
dtype: int64

In [14]:
tokyo_nodes.to_csv('network_inputs/tokyo_nodes.csv', index=False)
tokyo_edges.to_csv('network_inputs/tokyo_edges.csv', index=False)

In [ ]:
# check highways
tokyo_edges = pd.read_csv('network_inputs/tokyo_edges.csv')
highway = pd.read_csv('network_inputs/highway_edges.csv')
tokyo_edges_raw = pd.read_csv('network_inputs/tokyo_edges_osmnx_raw.csv')